# AES Key Recovery: 1st bit

## Description

In this attack, we aim to recover the first bit of the key using a binary classifier. We train a neural network on plaintext-ciphertext pairs as samples and the first bit of the key as labels.

## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from dataset.datasets import AESDatasetCiphertextPlaintextPairKey
from pipeline import *

2023-05-02 19:16:49.308352: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Importing the dataset

In [2]:
data = AESDatasetCiphertextPlaintextPairKey(128, 'large')

train_labels, train_samples, test_labels, test_samples = data.get_data()

Error: dataset/assets/moby-dick.txt
Error: dataset/assets/shakespeare.txt


In [3]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

===== Training Labels Shape: (1445705, 128)
===== Label Shape: (128,)
===== Training Samples Shape: (1445705, 256)
===== Sample Shape: (256,)
===== Testing Labels Shape: (619588, 128)
===== Testing Samples Shape: (619588, 256)


### Preprocessing

We add 1 preprocessing step:
As our goal is only to recover the first bit of the key, we only keep the first bit of the labels to train our network.


In [4]:
train_labels_1st_bit = np.array([l[0] for l in train_labels])
test_labels_1st_bit = np.array([l[0] for l in test_labels])

In [5]:
get_dataset_info(train_labels_1st_bit, train_samples, test_labels_1st_bit, test_samples)

===== Training Labels Shape: (1445705,)
===== Label Shape: ()
===== Training Samples Shape: (1445705, 256)
===== Sample Shape: (256,)
===== Testing Labels Shape: (619588,)
===== Testing Samples Shape: (619588, 256)


## Creating the model

In [6]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense, BatchNormalization, LayerNormalization
from keras.optimizers import Adam

### Model hyperparameters
In this code block, we specify most parameters and hyperparameters that will be used in the training of the neural network.

Add customization here.

In [12]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = 1

# units per hidden layer
units = 512

# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=0.001,
#     decay_steps=10000,
#     decay_rate=-0.9)

loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)
metrics = ['binary_accuracy']
epochs = 50
batch_size = 1000

### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

Add customization here.

In [13]:
# Type of model
neural_network = Sequential()

# Input layer
neural_network.add(Input(shape=input_shape))

# Hidden layers
#neural_network.add(BatchNormalization())
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))

# Output layer
neural_network.add(Dense(units=dim, activation='sigmoid'))

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_bce, metrics=metrics)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 512)               131584    
                                                                 
 dense_5 (Dense)             (None, 512)               262656    
                                                                 
 dense_6 (Dense)             (None, 512)               262656    
                                                                 
 dense_7 (Dense)             (None, 1)                 513       
                                                                 
Total params: 657,409
Trainable params: 657,409
Non-trainable params: 0
_________________________________________________________________


### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [14]:
history = train_model(neural_network, train_samples, train_labels_1st_bit, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.6936 - binary_accuracy: 0.4996
Epoch 2/50
1446/1446 [==============================] - 28s 19ms/step - loss: 0.6932 - binary_accuracy: 0.5003
Epoch 3/50
1446/1446 [==============================] - 27s 18ms/step - loss: 0.6932 - binary_accuracy: 0.5000
Epoch 4/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.6932 - binary_accuracy: 0.4999
Epoch 5/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.6932 - binary_accuracy: 0.5005
Epoch 6/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.6932 - binary_accuracy: 0.5014
Epoch 7/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.6932 - binary_accuracy: 0.5008
Epoch 8/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.6931 - binary_accuracy: 0.5022
Epoch 9/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.6931 - binary_accuracy: 0.5027
E

In [15]:
history = train_model(neural_network, train_samples, train_labels_1st_bit, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.6131 - binary_accuracy: 0.6086
Epoch 2/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.6105 - binary_accuracy: 0.6110
Epoch 3/50
1446/1446 [==============================] - 25s 17ms/step - loss: 0.6075 - binary_accuracy: 0.6131
Epoch 4/50
1446/1446 [==============================] - 25s 17ms/step - loss: 0.6044 - binary_accuracy: 0.6159
Epoch 5/50
1446/1446 [==============================] - 25s 18ms/step - loss: 0.6013 - binary_accuracy: 0.6180
Epoch 6/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.5985 - binary_accuracy: 0.6208
Epoch 7/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.5956 - binary_accuracy: 0.6227
Epoch 8/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.5930 - binary_accuracy: 0.6249
Epoch 9/50
1446/1446 [==============================] - 27s 18ms/step - loss: 0.5904 - binary_accuracy: 0.6266
E

In [16]:
history = train_model(neural_network, train_samples, train_labels_1st_bit, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.5244 - binary_accuracy: 0.6736
Epoch 2/50
1446/1446 [==============================] - 25s 17ms/step - loss: 0.5232 - binary_accuracy: 0.6744
Epoch 3/50
1446/1446 [==============================] - 25s 18ms/step - loss: 0.5221 - binary_accuracy: 0.6750
Epoch 4/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.5220 - binary_accuracy: 0.6751
Epoch 5/50
1446/1446 [==============================] - 25s 18ms/step - loss: 0.5202 - binary_accuracy: 0.6764
Epoch 6/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.5195 - binary_accuracy: 0.6768
Epoch 7/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.5189 - binary_accuracy: 0.6771
Epoch 8/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.5177 - binary_accuracy: 0.6779
Epoch 9/50
1446/1446 [==============================] - 26s 18ms/step - loss: 0.5181 - binary_accuracy: 0.6780
E

### Testing
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [17]:
results = test_model_binary(neural_network, test_samples, test_labels_1st_bit, batch_size)

620/620 [==============================] - 4s 7ms/step


In [20]:
print("Correct predictions: " + str(results["correct_predictions"]))
print("Accuracy: " + str(results["accuracy"]))

Correct predictions: 308967
Accuracy: 0.4986652420640813


In [21]:
save_model(neural_network, "aes_1st_bit")